In [6]:
import glob
from iso639 import Lang
import csv

txtFilters=glob.glob("./../../../data/filtered/alphabets/pud/*.txt")
csvsToAnalyze=glob.glob("./../../../data/non_filtered/corpora/pud/*.csv")

filterLang={}
for txt in txtFilters:
    with open(txt, newline='',encoding="utf-8") as txtOpened:
        pt3Name=txt.split("_")[1].split(".")[0]
        lines = txtOpened.readlines()
        newFilter=set()
        for line in lines:
            newFilter.add(line[0])
        filterLang[pt3Name]=newFilter

for csvfile in csvsToAnalyze:
    if  "jpn" not in csvfile or "zho" not in csvfile: 
        names=csvfile.split("\\")[1].split("_")[0]
        isoLan=names
        isoLanIdentifier=names
        with open("./../../../data/filtered/corpora/pud/"+isoLanIdentifier+"_pud.csv", 'w', encoding='UTF8', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(["","word","frequency","n_characters"])
            with open(csvfile, newline='',encoding="utf-8") as csvfileOpened:
                reader = csv.reader(csvfileOpened, delimiter=',', quotechar='"')
                next(reader)
                alphabetSet=filterLang[isoLanIdentifier]
                for row in reader:
                    valid=True
                    for letter in row[1]:
                        if letter not in alphabetSet:
                            valid=False
                            break
                    if valid:
                        writer.writerow(row)
